In [20]:
from tensorflow import keras, expand_dims
import numpy as np

In [21]:
# VisFin = keras.Sequential()
# VisFin.add(keras.layers.Conv2D(9, kernel_size = (9,9),padding='same', input_shape=(201, 43, 1), activation='relu'))
# VisFin.add(keras.layers.Conv2D(9, kernel_size = (9,9),padding='same', activation='relu'))
# VisFin.add(keras.layers.MaxPooling2D())
# VisFin.add(keras.layers.Conv2D(9, kernel_size = (6,6),padding='same', activation='relu'))
# VisFin.add(keras.layers.MaxPooling2D())
# VisFin.add(keras.layers.Flatten())
# VisFin.add(keras.layers.Dense(42, activation='relu'))
# VisFin.add(keras.layers.Dense(7,activation='softmax'))
# VisFin0 = keras.models.load_model('VisFin_v0.h5')
# VisFin1 = keras.models.load_model('VisFin_v1.h5')
# VisFin2 = keras.models.load_model('VisFin_v2.h5')
VisFin3 = keras.models.load_model('VisFin_v3.h5')

In [22]:
# VisFin.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
# categorical_crossentropy

In [23]:
from modules.data_feed import *
modeling = PrepareData('historic_data.db', 'returns')
a,b,c = modeling.make_graph('apple')


Connected to returns table!


In [24]:
# # TRAINING
# for tick in range(3):
#     data = modeling.mass_hacking(tick=tick)
#     tr_data = expand_dims(np.array(data[0]),axis=-1)
#     tr_key = np.array(data[1])
#     VisFin3.fit(tr_data, tr_key, epochs=10, verbose=2)
# VisFin3.save('VisFin_v3.h5')


In [25]:
# EVALUATE
for tick in range(5):
    data = modeling.mass_hacking(tick=tick)
    tr_data = expand_dims(np.array(data[0]),axis=-1)
    tr_key = np.array(data[1])
    # eval0 = VisFin0.evaluate(tr_data,tr_key, verbose=0)
    # eval1 = VisFin1.evaluate(tr_data,tr_key, verbose=0)
    # eval2 = VisFin2.evaluate(tr_data,tr_key, verbose=0)
    eval3 = VisFin3.evaluate(tr_data,tr_key, verbose=0)
    # print('VisFin0: ',eval0[0],eval0[1])
    # print('VisFin1: ',eval1[0],eval1[1])
    # print('VisFin2: ',eval2[0],eval2[1])
    print('VisFin3: ',eval3[0], eval3[1])

VisFin3:  1.1979341506958008 0.6294416189193726
VisFin3:  0.7056297659873962 0.7487437129020691
VisFin3:  0.06867163628339767 0.9949495196342468
VisFin3:  10.485092163085938 0.1354166716337204
VisFin3:  13.687849998474121 0.1666666716337204


In [26]:
# # convert to JS
import tensorflowjs
# tensorflowjs.converters.save_keras_model(VisFin0, 'js_format/VisFin_0')
# tensorflowjs.converters.save_keras_model(VisFin1, 'js_format/VisFin_1')
# tensorflowjs.converters.save_keras_model(VisFin2, 'js_format/VisFin_2')
tensorflowjs.converters.save_keras_model(VisFin3, 'js_format/VisFin_3')
